In [1]:
using Flux
using Flux.Tracker
using CSV: read
using BSON: @load, @save
using Missings: ismissing

In [ ]:
# Processing the data
f = CSV.read("./sandp500/all_stocks_5yr.csv")

T = 1259
Nw = 25                      # Window size
num = 1                      # starting number of data points
Nc = 0                       # number of companies
prev = f[1,7]                # first company in list
list = Array{Int64,1}(469)   # list of starting indicies of companies with 1259 data points

for i=2:length(f[:,7])
    
    # take average of adjacent values to account for missing values
    if ismissing(f[i,2]) && (f[i-1,7] == f[i+1,7])
        f[i,2] = (f[i-1,2] + f[i+1,2])
    end
    
    # count number of data points per company, adding beginning index
    # of each to list if number of data points == 1259
    if f[i,7] != prev
        prev = f[i,7]
        if num == 1259
            Nc += 1
            list[Nc] = i-num
        end     
        num = 1
    else
        num += 1
    end
end

# place each company with 1259 data points into its own colum
s_raw = Array{Float64, 2}(T,Nc)
for i=1:Nc
    s_raw[:,i] .= f[list[i]:list[i]+T-1,2]
end

In [ ]:
# reformat data for input to the model
t = Int64(floor(T/Nw))

s = Array{Float64, 2}(Nc*Nw,t)
for i=1:t
    s[:,i] .= vec(s_raw[Nw*(i-1)+1:Nw*i,:])
end

In [ ]:
function create()
    # common sub-expressions
    Ncw = Nc * Nw
    df1 = Int64(floor(Ncw / 3))
    df2 = Int64(floor(df1 / 6))
    
    # encoder weights saved to use in decoder weight creation
    W1a = Flux.glorot_uniform(df1,Ncw)
    W2a = Flux.glorot_uniform(df2,df1)
    
    layer1 = Dense(param(W1a),param(rand(df1)),relu)
    layer2 = Dense(param(W2a),param(rand(df2)),relu)
    layer3 = Dense(param(W2a'),param(rand(df1)),relu)
    layer4 = Dense(param(W1a'),param(rand(Ncw)),relu)
    
    m = Chain(layer1,layer2,layer3,layer4)
    return m
end

In [ ]:
function loss(model,s,λ)
    # loss is the mean squared error of the input with the output
    l = 0
    for i = 1:t
        l += Flux.mse(s[:,i],model(s[:,i]))
    end
    # regularization term
    penalty() = λ * (vecnorm(model.layers[1].W)^2 + vecnorm(model.layers[2].W)^ 2)
    return l + penalty()
end

In [ ]:
function update(model, α)
    # update weights
    model.layers[1].W.data .-= α * (model.layers[1].W.grad  + model.layers[4].W.grad')
    model.layers[4].W.data .-= α * (model.layers[1].W.grad' + model.layers[4].W.grad )
    model.layers[2].W.data .-= α * (model.layers[2].W.grad  + model.layers[3].W.grad') 
    model.layers[3].W.data .-= α * (model.layers[2].W.grad' + model.layers[3].W.grad )
    
    # update biases
    model.layers[1].b.data .-= α * model.layers[1].b.grad
    model.layers[2].b.data .-= α * model.layers[2].b.grad
    model.layers[3].b.data .-= α * model.layers[3].b.grad
    model.layers[4].b.data .-= α * model.layers[4].b.grad
    
    # for each layer
    for layer in model.layers
        # set grads to 0
        layer.W.grad .= 0
        layer.b.grad .= 0
    end
end

In [ ]:
function train(model, s, loss_f)
    # calculate loss
    l = loss(model, s, 0.1)
    # calculate gradients
    back!(l)
    # perform updates
    update(model, 0.1)
    return l
end

In [ ]:
model = create()

for i=1:2
    l = train(model, s, loss)
    if i%2==0
        println("Iteration: $i\n   loss: $l")
    end
end

In [ ]:
# save model
weights = Tracker.data.(params(model))
@save "model.bson" model
@save "model_weights.bson" weights

In [ ]:
# load model
@load "model.bson" model
@load "model_weights.bson" weights
Flux.loadparams!(model, weights)

# test that model loaded corectly
l = loss(model, s, 0.1)
println("loss: $l")